In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda', index=0)

In [3]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")

In [4]:
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")
model.to(device);

In [5]:
sum(p.numel() for p in model.parameters())

774030080

### Конфиг из телеграм бота
Бот: https://github.com/polakowo/gpt2bot  
Конфиг:  https://github.com/polakowo/gpt2bot/blob/b7a202352e4b423af685784f61987f9260e89cf5/chatbot.cfg  


In [6]:
def decode_answer(model_output, input_text_len):
    answer = tokenizer.decode(model_output[input_text_len:], skip_special_tokens=True)
    return answer


def print_samples(question, top_p=1.0, top_k=40, temperature=0.7, num_beams=1,
            repetition_penalty=1.0, length_penalty=1.0, no_repeat_ngram_size=0, num_return_sequences=5, repeat=1):
    input_text = question
    input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='pt').to(device)
    for _ in range(repeat):
        sample_outputs = model.generate(
            input_ids,
            do_sample=True, 
            early_stopping=False,
            max_length=128,
            temperature=temperature,
            top_k=top_k, 
            top_p=top_p,
            num_beams=num_beams,
            repetition_penalty=repetition_penalty,
            length_penalty=length_penalty,
            no_repeat_ngram_size=no_repeat_ngram_size,
            num_return_sequences=num_return_sequences,
            pad_token_id=tokenizer.eos_token_id
        )
        answers = sorted(map(lambda text: decode_answer(text, input_ids.shape[-1]-1), sample_outputs),
                         key=len, reverse=True)
        for answer in answers:
            print(answer)
            print('**********\n')

### Примеры из репозитория
https://github.com/microsoft/DialoGPT  

In [7]:
print_samples('Does money buy happiness?')

No, but I think it's the only thing the people of that country are happy about.
**********

It does if you're a politician.
**********

No, but it can buy happiness.
**********

If it doesn't, it will.
**********

It doesn't.
**********



In [8]:
print_samples('who is the first president of the United States')

George Washington
**********

Lincoln, right?
**********

Santino Marella
**********

Cory Booker
**********

You are
**********



In [9]:
print_samples('who is the first president of the United States?')

I don't know, I think it's an American President.
**********

I think its the United States President
**********

I think it was Abraham Lincoln
**********

Ronald Reagan
**********

You are.
**********



In [10]:
print_samples('what is the boiling point of water?')

I know I was looking for that for a while.
**********

The boiling point of water is water...
**********

It's the boiling point of water, duh.
**********

The boiling point of water is 12.8.
**********

About 60 degrees F
**********



In [11]:
print_samples('which one is bigger, sun or moon?')

Sun is bigger but moon gets bigger over time.
**********

Moon. That's why the moon is so big.
**********

Sun. It's a bigger solar system.
**********

Moon, but only on the surface.
**********

Sun, because it's a satellite.
**********



In [12]:
print_samples('which animal has black and white stripes?')

The black and white stripes are the stripes on the side of the animal, the white stripes are the stripes on the back of the animal.
**********

The black and white stripes are not in the original design, and the cat's stripes are in the original design.
**********

The one that looks like a turtle.
**********

Black and white is a type of fur.
**********

The one that's black and white.
**********



In [13]:
print_samples('what is the meaning of life ?')

This is the real question we need to ask
**********

Why is the sky blue?
**********

To be satisfied
**********

To be a woman
**********

You know.
**********



In [14]:
print_samples('who won the world cup in 2018 ?')

Gerrard, but the team of the year award was given to Suarez
**********

England. They won the Euros in 2016, 2017, 2018, and 2019.
**********

The world's next greatest team.
**********

England and Argentina
**********

England, obviously
**********



In [15]:
print_samples("""Nvidia's Titan RTX is really good .""")

I looked it up and it looks awesome!
**********

How does it compare to the Titan X?
**********

I really like the graphics of it.
**********

That card looks awesome.
**********

Second this.
**********



In [16]:
print_samples("""Nvidia's Titan RTX is really good.""")

Not really, it's got a pretty bad screen to make it a good experience.
**********

Yeah, it's definitely up there. The other two are pretty good.
**********

That's the one I'm currently working on.
**********

Agreed, especially the 1080ti version
**********

I know. It's a great card.
**********



In [17]:
print_samples("""Can Porsche beat Tesla with its new Taycan EV ?""")

Why buy a taycan when you can buy a Porsche?
**********

I think we can all agree on that point.
**********

Nah, the Tesla is a lot more advanced.
**********

With their new supercharger, yes.
**********

Well they have a 100k car.
**********



In [18]:
print_samples("""Can Porsche beat Tesla with its new Taycan EV?""")

TIL Teslas are built on the same process and are the same tech as normal cars.
**********

With the new Teslas, you should be able to beat any car on the road.
**********

No, but they can't beat Tesla with their new Teslas.
**********

I would like to see a porsche drive a tesla.
**********

Porsche already has a tesla.
**********



In [19]:
print_samples("""What do you think of the DialoGPT repo ?""")

I like it, it's not exactly a package though it's a package you can install without having to do anything.
**********

It's really good. I've never used anything else so I'll be looking at it.
**********

I am not familiar with it, but I am familiar with the other ones.
**********

I'm not familiar with it. Is it a package or a binary change?
**********

I like it a lot, for what I use it for.
**********



In [20]:
print_samples("""What do you think of the DialoGPT repo?""")

I haven't used it yet, but I think it looks pretty good. I'm using the default ROM and I don't really have any custom ROMs installed.
**********

I used it a while ago but I don't like the look of it. I would use the default launcher and the icon pack, personally.
**********

Dialog is great. It's just the one that doesn't work for me.
**********

Love it. Thanks for the recommendation.
**********

I'll try it out, thanks!
**********



### Примеры из AskReddit

In [21]:
print_samples("""What's the grossest thing you've ever tasted?""")

I think the last meal I had was a steak steak steak steak steak steak steak steak... Steak.
**********

I had some really gross stuff when I was younger. I can't remember much, though.
**********

I would like to know this as well.
**********

I think I'm on to something here.
**********

I like the smell of a banana.
**********



In [22]:
print_samples("""If you could have 1 superpower what would it be and why?""")

Teleportation. I think it would be pretty cool to run around the city like some sort of ninja.
**********

I'd like to live on the moon so I can play some video games.
**********

Speedforce, because it's a reality warper.
**********

Speedforce. Because it's my superpower.
**********

Teleporter. I'd be a great teleporter.
**********



In [23]:
print_samples("""Is there anything you should be doing right now, but you're wasting your time on reddit instead? If so, what is it?""")

Not really. This whole internet experiment has been pretty fun.
**********

I have no idea what you're talking about.
**********

Not a whole lot. Just reading the news.
**********

No, just making a comment.
**********

I haven't decided yet.
**********



In [24]:
print_samples("""What is the scariest/creepiest theory you know about?""")

I think it was about how we can't really trust people to not be asexual.
**********

That this is actually just a metaphor for human consciousness.
**********

No, the scientific definition of scariest is not a thing.
**********

I just want to know the answers to the questions above.
**********

The one that involves a goat.
**********



In [25]:
print_samples("""In the John Wick universe, assassins are shown to be everyone and everywhere, thus indicating a buyer's market. What limited time deals and offers do assassins provide to out price the competition? How does the economics of the world operate with so many trained killers?""")

It's not like the assassins want to be assassins. It's like they want to be a mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary mercenary
**********

Like any other market, the more common person will buy the more common the discount will be.And if the discount is too good a deal, they'll cancel the order.
**********

Assassins are more like assassins in terms of their skill and the necessity of a living.
**********

What do you mean by this?
**********

They don't?
**********



In [26]:
print_samples("""What is you favorite sex scene in the Bible?""")

I think I'm going to say the part with the two kings. I know the names, but that's too long for me to remember.
**********

The one where Jesus comes riding with his Apostles is my favorite.
**********

The ones that aren't based on actual biblical rules.
**********

The one where Jesus is a woman.
**********

I don't really know many.
**********



In [27]:
print_samples("""What are underrated websites and what do you use them for?""")

I use them to play with people I know and to play with people I know.
**********

I have an addiction. I use them for stuff like this.
**********

I was not aware that there were more than one.
**********

Curious about this as well.
**********

I use them for everything.
**********



In [28]:
print_samples("""Why do you hate the people that you hate?""")

We're just jealous of their ability to be jealous.
**********

Why do you hate the people who you hate?
**********

because it's the internet
**********

Because they hate me.
**********

Why do you hate me?
**********



In [29]:
print_samples("""People who used to work in photo labs before digital cameras, what weird stuff did you develop?""")

I used to work in a large glass and digital photo lab. We used to have to develop a bunch of photos to get them to look good.
**********

I have been in photo labs for over a decade and this is my first time seeing this.
**********

I was the photographer and the guy who developed the film!
**********

You know, my guess is that he developed a digital camera.
**********

I'm pretty sure digital cameras were all digital then.
**********



In [30]:
print_samples("""To the people who check behind the shower curtain before using the washroom; what’s the next plan if you see someone?""")

Just take the shower out and say it's been used once.
**********

Just say sorry to the people who did the same thing
**********

Dont tell them you want them to know.
**********

I think the plan is to not be seen.
**********

Just tell them to leave it alone.
**********



In [31]:
print_samples("""If you came with a warning label, what would it be?""")

I think I read it had a warning label.
**********

A warning label I guess
**********

It's a warning label.
**********

A warning label?
**********

Mystery solved
**********



In [32]:
print_samples("""What is the best decision you've made while drunk?""")

I don't know. I guess the answer to that would be obvious.
**********

I don't know, and I don't know if I want to know.
**********

Probably to post this in the first place.
**********

I went to a bar in Seattle.
**********

I drank way, way too much.
**********



In [33]:
print_samples("""What’s one rule you live by?""")

I have a rule that I never want to be in a game with a player on the other team.
**********

I'm telling you, you've got a lot of'em.
**********

The one with the highest power
**********

I think it's the second rule.
**********

This is a very good question!
**********

